analisando Dataset de celulares, nesse exemplo vou usar o pandas para entende do que se tratar o cnoteúdo sem usar referência externa

## o arquivo foi baixado do kaggle

In [ ]:
import pandas as pd

import chardet

with open("/content/Mobiles Dataset (2025).csv", 'rb') as f:
    result = chardet.detect(f.read())
print(result['encoding'])

df = pd.read_csv("/content/Mobiles Dataset (2025).csv", encoding=result['encoding'])

ISO-8859-1


foi indentificado que o arquivo possui uma codificação diferente do utf-8

In [ ]:
print(df.head()) # lendo o cabeçalho
print("\n---------------------------------")
print(df.describe()) #lendo a descrição
print("\n---------------------------------")
print(df.info()) #lendo a informação

  Company Name            Model Name Mobile Weight  RAM Front Camera  \
0        Apple       iPhone 16 128GB          174g  6GB         12MP   
1        Apple       iPhone 16 256GB          174g  6GB         12MP   
2        Apple       iPhone 16 512GB          174g  6GB         12MP   
3        Apple  iPhone 16 Plus 128GB          203g  6GB         12MP   
4        Apple  iPhone 16 Plus 256GB          203g  6GB         12MP   

  Back Camera   Processor Battery Capacity Screen Size  \
0        48MP  A17 Bionic         3,600mAh  6.1 inches   
1        48MP  A17 Bionic         3,600mAh  6.1 inches   
2        48MP  A17 Bionic         3,600mAh  6.1 inches   
3        48MP  A17 Bionic         4,200mAh  6.7 inches   
4        48MP  A17 Bionic         4,200mAh  6.7 inches   

  Launched Price (Pakistan) Launched Price (India) Launched Price (China)  \
0               PKR 224,999             INR 79,999              CNY 5,799   
1               PKR 234,999             INR 84,999              

pelo resultado podemos analisa que se tratar de uma listas de smartphones que foram lançados em 2024 mas podemos fazer uma analise profunda porém devemos verificar a qualidade dos dados , cehcando se tem nulos, redudantes e faltantes

In [ ]:
# Mostra a quantidade de valores nulos em cada coluna
print(df.isnull().sum())

# Mostra a porcentagem de valores nulos em cada coluna
print(df.isnull().mean() * 100)

Company Name                 0
Model Name                   0
Mobile Weight                0
RAM                          0
Front Camera                 0
Back Camera                  0
Processor                    0
Battery Capacity             0
Screen Size                  0
Launched Price (Pakistan)    0
Launched Price (India)       0
Launched Price (China)       0
Launched Price (USA)         0
Launched Price (Dubai)       0
Launched Year                0
dtype: int64
Company Name                 0.0
Model Name                   0.0
Mobile Weight                0.0
RAM                          0.0
Front Camera                 0.0
Back Camera                  0.0
Processor                    0.0
Battery Capacity             0.0
Screen Size                  0.0
Launched Price (Pakistan)    0.0
Launched Price (India)       0.0
Launched Price (China)       0.0
Launched Price (USA)         0.0
Launched Price (Dubai)       0.0
Launched Year                0.0
dtype: float64


In [ ]:
# Retorna o número de linhas duplicadas
print(df.duplicated().sum())
print("\n-------------------------")

# Mostra todas as linhas duplicadas
print(df[df.duplicated(keep=False)])

15

-------------------------
    Company Name        Model Name Mobile Weight   RAM Front Camera  \
344         Oppo          A3 128GB          186g   4GB          5MP   
347         Oppo          A3 128GB          186g   4GB          5MP   
423         Oppo        K11x 128GB          195g   8GB         16MP   
424         Oppo        K11x 256GB          195g  12GB         16MP   
425         Oppo        K10x 128GB          195g   6GB         16MP   
426         Oppo        K10x 256GB          195g   8GB         16MP   
427         Oppo      K10 5G 128GB          205g   8GB         16MP   
428         Oppo         K9x 128GB          194g   6GB         16MP   
429         Oppo         K9x 256GB          194g   8GB         16MP   
430         Oppo   K9 Pro 5G 128GB          180g   8GB         16MP   
431         Oppo   K9 Pro 5G 256GB          180g  12GB         16MP   
432         Oppo       K9 5G 128GB          172g   8GB         32MP   
433         Oppo       K9 5G 256GB          172

In [ ]:
# Mostra os valores únicos de uma coluna (útil para categóricos)
# print(df['coluna'].unique())

#como são dados de hardware os valores costuma se duplicados então provavelmente não existe únicos


KeyError: 'coluna'

agora iremos fazer uma filtragem em 3 colunas para analise

In [19]:
#print(df["Launched Price (USA)"].dtype)

#print(df.columns.tolist())  # Lista todas as colunas disponíveis

# Função para limpar e converter colunas de preço
def clean_price_column(column):
    return (
        column.astype(str)
        .str.replace(r'[^\d.]', '', regex=True)  # Remove tudo exceto números e ponto
        .astype(float)
    )

# Limpa e converte as colunas de preço
df["Launched Price (USA)"] = clean_price_column(df["Launched Price (USA)"])
df["Launched Price (India)"] = clean_price_column(df["Launched Price (India)"])
df["Launched Price (China)"] = clean_price_column(df["Launched Price (China)"])

# Filtra linhas onde o preço nos EUA, Índia ou China seja maior que 500
df_filtered = df[
    (df["Launched Price (USA)"] > 500) |
    (df["Launched Price (India)"] > 42642.54) |
    (df["Launched Price (China)"] > 3643.5)
]

# Seleciona apenas as colunas desejadas (incluindo os novos países)
df_filtered = df_filtered[[
    "Company Name",
    "Model Name",
    "Launched Price (USA)",
    "Launched Price (India)",
    "Launched Price (China)"
]]

# Mostra as primeiras linhas
print(df_filtered.head())

  Company Name            Model Name  Launched Price (USA)  \
0        Apple       iPhone 16 128GB                 799.0   
1        Apple       iPhone 16 256GB                 849.0   
2        Apple       iPhone 16 512GB                 899.0   
3        Apple  iPhone 16 Plus 128GB                 899.0   
4        Apple  iPhone 16 Plus 256GB                 949.0   

   Launched Price (India)  Launched Price (China)  
0                 79999.0                  5799.0  
1                 84999.0                  6099.0  
2                 89999.0                  6499.0  
3                 89999.0                  6199.0  
4                 94999.0                  6499.0  


pelo resultado da saída os Iphones vendidos ano de 2024 nos USA acima de 500 dolares são parecidos do que em outros paises onde eu usei um converso externo de moeda para parear

In [20]:
iphones = df[df["Model Name"].str.contains("iPhone 16")]
print(iphones[["Model Name", "Launched Price (USA)", "Launched Price (India)", "Launched Price (China)"]])

                 Model Name  Launched Price (USA)  Launched Price (India)  \
0           iPhone 16 128GB                 799.0                 79999.0   
1           iPhone 16 256GB                 849.0                 84999.0   
2           iPhone 16 512GB                 899.0                 89999.0   
3      iPhone 16 Plus 128GB                 899.0                 89999.0   
4      iPhone 16 Plus 256GB                 949.0                 94999.0   
5      iPhone 16 Plus 512GB                 999.0                104999.0   
6       iPhone 16 Pro 128GB                 999.0                 99999.0   
7       iPhone 16 Pro 256GB                1049.0                104999.0   
8       iPhone 16 Pro 512GB                1099.0                114999.0   
9   iPhone 16 Pro Max 128GB                1099.0                109999.0   
10  iPhone 16 Pro Max 256GB                1199.0                114999.0   
11  iPhone 16 Pro Max 512GB                1299.0                124999.0   

após uma segunda analise mais robusta aparenta que na india é um pouco mais caro entorno de 20% em comparação aos USA porém vamos analisa agora a %

In [21]:
# Calcular diferenças percentuais
df['India Premium (%)'] = ((df['Launched Price (India)']/83 - df['Launched Price (USA)'])/df['Launched Price (USA)']*100).round(1)
df['China Discount (%)'] = ((df['Launched Price (USA)'] - df['Launched Price (China)']/7.2)/df['Launched Price (USA)']*100).round(1)
print(df[['Model Name', 'Launched Price (USA)', 'India Premium (%)', 'China Discount (%)']])

               Model Name  Launched Price (USA)  India Premium (%)  \
0         iPhone 16 128GB                 799.0               20.6   
1         iPhone 16 256GB                 849.0               20.6   
2         iPhone 16 512GB                 899.0               20.6   
3    iPhone 16 Plus 128GB                 899.0               20.6   
4    iPhone 16 Plus 256GB                 949.0               20.6   
..                    ...                   ...                ...   
925          Pad 5G 128GB                 280.0                3.3   
926          Pad 5G 256GB                 300.0                4.4   
927  Galaxy Z Fold6 256GB                1899.0                4.7   
928  Galaxy Z Fold6 512GB                1719.0               24.1   
929    Galaxy Z Fold6 1TB                2259.0                7.2   

     China Discount (%)  
0                  -0.8  
1                   0.2  
2                  -0.4  
3                   4.2  
4                   4.9  
.. 

essa última analise levar a creer que na china tem preço equivalentes ou abaixo , india a cima em comparação aos USA